In [ ]:
import os

try:
    if CWD_UPDATED:
        pass
except:
    os.chdir("..")

CWD_UPDATED = True

print(f"Working directory: {os.getcwd()}")

In [ ]:
from components.roll import Roll
from components.note import Note
from components.chord import Chord, ChordQuality
from transformers.pitch import Pitch
from transformers.interval import Interval

In [ ]:
def add_as_stride(roll: Roll, chord: Chord, start: float, duration: float):
    for i, time in enumerate(range(start, start + duration)):
        if i % 8 == 0:
            roll.add_notes(
                Note(
                    pitch=chord.root,
                    start=time,
                    duration=roll.Time(1 / 8),
                ).reoctave_near_pitch(Pitch("C3"))
            )
        elif i % 8 == 4:
            roll.add_notes(
                Note(
                    pitch=chord.root + Interval("5"),
                    start=time,
                    duration=roll.Time(1 / 8),
                ).reoctave_near_pitch(
                    min(roll.get_pitches_at_time(time - roll.Time(1 / 4)).set),
                    position="below",
                )
            )
        elif i % 4 == 2:
            roll.add_notes(
                *[
                    Note(
                        pitch=pitch,
                        start=time,
                        duration=roll.Time(1 / 16),
                    ).reoctave_near_pitch(Pitch("C5"))
                    for pitch in chord.set
                ]
            )

In [ ]:
key = Pitch("C")

I = Chord(key, ChordQuality.Maj)
IV = Chord(key + Interval("4"), ChordQuality.Maj)
V = Chord(key + Interval("5"), ChordQuality.Maj)

roll = Roll(bpm=110)

add_as_stride(roll, I, roll.Time(0), roll.Time(0.5))
add_as_stride(roll, IV.get_V7(), roll.Time(0.5), roll.Time(0.5))
add_as_stride(roll, IV, roll.Time(1), roll.Time(0.5))
add_as_stride(roll, V.get_V7(), roll.Time(1.5), roll.Time(0.5))
add_as_stride(roll, V, roll.Time(2), roll.Time(1))
add_as_stride(roll, I, roll.Time(3), roll.Time(0.5))
roll.add_notes(
    Note(I.root, roll.Time(3.5), roll.Time(0.25)).reoctave_near_pitch(Pitch("C3"))
)

In [ ]:
os.makedirs("playground/output", exist_ok=True)
with open("playground/output/output.mid", "wb") as fout:
    roll.to_midi().writeFile(fout)

In [ ]:
os.system("open -a VLC playground/output/output.mid")